In [1]:
# Install a pip package in the current Jupyter kernel
import sys


### General
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors
#from scipy.optimize import curve_fit
from tqdm import tqdm
import pandas as pd

### Query
from astroquery.sdss import SDSS

# 01 - Data Processing

In [2]:
df = pd.read_csv("data/sdssdr14-wd-catalog/Table2.csv")

ps = []
ms = []
fs = []

for i in range(len(df['P-M-F'])):
    tpmf = df['P-M-F'][i].split('-')
        
    ps.append(int(tpmf[0]))
    ms.append(int(tpmf[1]))
    fs.append(int(tpmf[2]))
        
df['plate'] = ps
df['mjd'] = ms
df['fiber'] = fs

C:\Users\arsen\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (6,7,8,9,10,11,12,13,14,15,16) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
print(type(df['SNg'][2]))

<class 'numpy.int64'>


In [4]:
from astropy.coordinates import SkyCoord
from astropy import units as u

q = df['SNg'].tolist().index(max(df['SNg']))

dothing = True

if dothing:
    xids = []
    
    for i in tqdm (range(len(df['SNg']))):
        try:
            xid = SDSS.get_spectra_async(plate=df['plate'][i], fiberID=df['fiber'][i], mjd=df['mjd'][i])
            xids.append(xid)
        except:
            pass
else:
    xid = SDSS.get_spectra_async(plate=df['plate'][i], fiberID=df['fiber'][i], mjd=df['mjd'][i])
    xids = None
    
print(xid)
    
#url = str(xid[0]).split(' ')
#print(url)
#print((str(xids[0]).split(' ')[4]).split('-'))

from numpy import asarray
from numpy import save

if dothing:
    xids2 = np.array(xids)
    xids2.tofile('data/wd-spectra.txt', sep = ',')
    
print(len(xids))

 57%|██████████████████████████████████████████▏                               | 21322/37406 [00:15<00:11, 1404.97it/s]C:\Users\arsen\anaconda3\lib\site-packages\astroquery\sdss\core.py:874: VisibleDeprecationWarning: Reading unicode strings without specifying the encoding argument is deprecated. Set the encoding, use None for the system default.
  arr = np.atleast_1d(np.genfromtxt(io.BytesIO(response.content),
100%|██████████████████████████████████████████████████████████████████████████| 37406/37406 [00:28<00:00, 1314.25it/s]

[Downloaded object from URL https://data.sdss.org/sas/dr14/sdss/spectro/redux/26/spectra/1000/spec-1000-52643-0512.fits with ID 3024688088352]
37406



C:\Users\arsen\AppData\Local\Temp/ipykernel_900/682111842.py:31: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  xids2 = np.array(xids)


In [5]:
print(df)

                 P-M-F               RA-DEC  SNg       u     su       g  \
0      0266-51602-0314  094107.47-001949.70   12  19.369  0.023  18.965   
1      0266-51630-0015  094739.30-003840.13   20  19.405  0.029  18.104   
2      0266-51630-0026  094901.28-001909.61   40  16.951  0.010  16.511   
3      0266-51630-0031  094804.30-000738.16   15  18.688  0.023  18.278   
4      0266-51630-0037  094917.06-000023.67   11  19.271  0.024  18.796   
...                ...                  ...  ...     ...    ...     ...   
37401  8958-57514-0708  103122.68+255738.03   38  18.129  0.018  17.242   
37402  8958-57514-0794  103255.79+261139.66   38  18.245  0.018  17.391   
37403  9003-57488-0602  102914.09+254714.37   39  18.358  0.020  17.523   
37404  9003-57488-0700  103012.64+250954.63   42  18.026  0.018  17.156   
37405  1000-52643-0512  104217.62+064907.37   12  19.414  0.032  18.507   

          sg       r     sr       i  ...       z     sz    ebv    ppm      l  \
0      0.014  19.06

In [6]:
print(min(df['g']))
print(max(df['g']))

0.0
137.0


In [36]:
query1 = "select s.bestObjID, s.specObjID, s.plate, s.mjd, s.fiberID \
    from SpecObj as s \
    where s.plate > 265\
    and s.plate < 3400\
    and s.class = \'STAR\'"
query2 = "select s.bestObjID, s.specObjID, s.plate, s.mjd, s.fiberID \
    from SpecObj as s \
    where s.plate > 3400\
    and s.plate < 9004\
    and s.class = \'STAR\'"
            
white_dwarf1= SDSS.query_sql(query1)
white_dwarf2= SDSS.query_sql(query2)


In [37]:
from astropy.table import Table, vstack

twhite_dwarf = vstack([white_dwarf1, white_dwarf2])
white_dwarf = twhite_dwarf
print(len(twhite_dwarf))

887342


In [38]:
id_tuple = np.array([df['plate'], df['mjd'], df['fiber']]).T
print([2699,54414,525] in id_tuple)
white_dwarf = twhite_dwarf.to_pandas()

print(type(white_dwarf))
i = np.where((id_tuple[:,2] == 360) & (id_tuple[:,0] == 54414))

True
<class 'pandas.core.frame.DataFrame'>


In [39]:
def search(plate, fiber, mjd):
    urls = []
    drops = []
    
    for i in tqdm(range(len(plate))):
        notfound = False
        j = np.where((id_tuple[:,0] == plate[i]) & (id_tuple[:,1] == mjd[i]) & (id_tuple[:,2] == fiber[i]))
        
        try:
            k = j[0][0]
        except:
            notfound = True
            
        if not notfound: 
            try:
                urls.append(str(xids[k][0]).split(' ')[4])
            except:
                notfound = True
        if notfound:
            drops.append(i)
            
    return drops, urls

drops, urls = search(white_dwarf['plate'], white_dwarf['fiberID'], white_dwarf['mjd'])


100%|███████████████████████████████████████████████████████████████████████| 887342/887342 [01:28<00:00, 10041.21it/s]


In [40]:
print(len(drops))
print(len(white_dwarf) - len(df))

white_dwarf = white_dwarf.drop(drops)

print(len(white_dwarf))

white_dwarf['url'] = urls
white_dwarf.reset_index(inplace=True, drop=True)
print(white_dwarf[:10])

855752
849936
31590
             bestObjID     specObjID  plate    mjd  fiberID  \
0  1237664087534732151  3.038948e+18   2699  54414      525   
1  1237664087534797369  3.038930e+18   2699  54414      459   
2  1237664087534928362  3.038958e+18   2699  54414      561   
3  1237664089146064917  1.415271e+18   1257  52944       53   
4  1237664089146064948  1.415269e+18   1257  52944       47   
5  1237664089146065069  1.415257e+18   1257  52944        1   
6  1237664089682149405  1.415322e+18   1257  52944      240   
7  1237664089682149414  1.415318e+18   1257  52944      223   
8  1237664089682149435  1.415319e+18   1257  52944      228   
9  1237664089682346088  1.415308e+18   1257  52944      190   

                                                 url  
0  https://data.sdss.org/sas/dr14/sdss/spectro/re...  
1  https://data.sdss.org/sas/dr14/sdss/spectro/re...  
2  https://data.sdss.org/sas/dr14/sdss/spectro/re...  
3  https://data.sdss.org/sas/dr14/sdss/spectro/re...  
4  https://

In [41]:
white_dwarf.to_csv('data/white_dwarves.csv')

## 02 -- Spectra
---
No more data after here

In [ ]:
from astropy.io import fits

url = white_dwarf['fits'][q]

star = fits.open(url)

star[0].header['SPEC_ID']

In [ ]:
import sdss

specObjID = tuple(white_dwarf['objectID'])
print(type(specObjID[0]))

sql = "SELECT * \
    FROM SpecObj AS s \
    WHERE s.specObjID IN {} \
    ".format(specObjID[:100])

df2 = SDSS.query_sql(sql)
print(df2)

In [ ]:
##### https://www.sdss.org/dr17/spectro/spectro_basics/
# https://data.sdss.org/datamodel/files/SPECTRO_REDUX/RUN2D/PLATE4/spCFrame.html

header = star[1].header
data = star[1].data

wavelength = 10**data['loglam']
flux = data['flux']
ivar = data['ivar']

plt.figure(figsize=(20,8))

plt.plot(wavelength, flux)
plt.grid()
plt.ylabel(r'Flux [$10^{-17}$ ergs/s/cm2/A]')
plt.xlabel(r'Wavelength [A]')
plt.title('White Dwarf Light Curve')
ax = plt.gca()

plt.show()

In [ ]:
from lmfit.models import GaussianModel, ConstantModel

balmer_lines = [6562.79, 4861.35, 4340.47, 4101.73, 3970.07, 3889.06, 3835.40]
c = 299792458 #m/s

# n between 1 & 7
n = 7

lambda_obs = []
dlambda_obs = []

for i in range(n):
    line = balmer_lines[i]
    print(str(i) + ': ' + str(line))
    
    ha = np.where(np.logical_and([wavelength > line - 0.1 * line], [wavelength < line + 0.1 * line]))[1]
    
    x = wavelength[ha]
    y = flux[ha]
    
    model = (ConstantModel(prefix='offset_') +
             GaussianModel(prefix='p_'))
    
    params = model.make_params(offset_c=max(wavelength[ha]), p_amplitude=(min(wavelength[ha])-max(wavelength[ha])), _sigma=100, p_center=line)
    init = model.eval(params, x=x)
    result = model.fit(y, params, x=x)
    
    lambda_obs.append(result.params['p_center'].value)
    dlambda_obs.append(result.params['p_center'].stderr)

v_app = 0
dv_app = 0
    
for i in range(n):
    v_app += (c * ((lambda_obs[i] - balmer_lines[i]) / balmer_lines[i]) * 1e-3) / n
    dv_app = np.abs(v_app * 1e3) * np.sqrt(np.sum((np.array(dlambda_obs) / np.array(lambda_obs))**2) ) / n
    
print(str(v_app) + '+/-' + str(dv_app))

plt.figure(figsize=(15,6))

plt.plot(wavelength[ha], flux[ha], label='Spectrum Data')
plt.plot(wavelength[ha], result.best_fit, c='black', label='Gaussian Fit')
plt.xlim((line - 0.1 * line, line + 0.1 * line))
#plt.ylim((150,400))
plt.grid()
plt.ylabel(r'Flux [$10^{-17}$ ergs/s/cm2/A]')
plt.xlabel(r'Wavelength [A]')
plt.title(r'White Dwarf $H\alpha$ Absorbtion Line')
ax = plt.gca()

plt.legend()
plt.show()

In [ ]:

'''
lambda_obs = result.params['p_center'].value
dlambda_obs = result.params['p_center'].stderr

v_app = c * ((lambda_obs - h_alpha) / h_alpha) * 1e-3
dv_app = np.abs(v_app * 1e3) * np.sqrt((dlambda_obs / lambda_obs)**2 )

print('Observed speed by redshift: %.2f +/- %.2f km/s' % (v_app, dv_app))
'''

In [ ]:
from astropy.io import fits
import matplotlib.pyplot as plt
import numpy as np
#!{sys.executable} -m pip install tensorflow


import sys
sys.path.append('../wdtools') # Your path should contain the parent GitHub repository

In [ ]:
import wdtools

gfp = wdtools.GFP(resolution = 2, specclass = 'DA')

labels, e_labels, redchi = gfp.fit_spectrum(wavelength, flux, ivar,
                                mcmc = True, nwalkers = 25, burn = 50, ndraws = 50,
                                plot_init = True, make_plot = True, plot_corner = True, savename = 'example',
                                verbose = True,
                                lines = ['beta', 'gamma', 'delta', 'eps', 'h8'])

In [ ]:
sp = wdtools.SpecTools()

h_alpha = 6562.79

rv, e_rv = sp.get_line_rv(wavelength, flux, ivar, centroid = h_alpha,
                          distance = 100, edge = 15,
                          nmodel = 2, plot = True)

In [ ]:
print('Observed speed by redshift (WDTools): %.2f +/- %.2f km/s' % (rv, e_rv))
print('Observed speed by redshift (Gaussian): %.2f +/- %.2f km/s' % (v_app, dv_app))

In [ ]:
print('Teff = %i ± %i Kelvin' % (labels[0], e_labels[0]))
print('logg = %.2f ± %.2f log[cm/s^2]' % (labels[1], e_labels[1]))
print('RV = %.1f ± %.1f km/s' % (labels[2], e_labels[2]))
print('reduced chi^2 = %.2f' % (redchi))